#Примерная модель

In [1]:
!pip install xgboost
!pip install imbalanced-learn


import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier


from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
df = pd.read_csv('df_for_train.csv')
df.dropna(inplace=True)

,name,text,keywords,type
0,855c8450351c98a25ea2ac22a0256c7f,г договор г москва г нижеподписавшиеся граждан...,аренды квартиры договора аренды получает време...,4
1,35b6a0f57d909507c5aa9a8972b15f35,приложение приказу ав договор возмездного оказ...,оказания услуг настоящего договора оказать усл...,2
2,6babd660f1a9192017a06b7e93bea919,договор г москва общество ограниченной ответст...,настоящего договора настоящему договору сдачи ...,3
3,2b408919fd9833dc3f6892aad753f21f,договор подряда номердокумента датадокумента н...,настоящего договора сдачи приемки настоящему д...,3
4,214d620d9c54bc83111277dd872d3cb2,договор поставки поставка указывается предмет ...,актами российской момента заключения правовыми...,1


In [5]:
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(df['text'])
df1 = pd.DataFrame(data=feature_matrix.todense(), columns=tfidf.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
X = df1
y = df['type']

In [9]:
df['type'].value_counts()

2    28
4    23
5    23
3    21
1    21
Name: type, dtype: int64

In [11]:
X_new = SelectKBest(chi2, k=70).fit_transform(X, y)
X_new.shape

(116, 70)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [13]:
X_resampled, y_resampled = SVMSMOTE(random_state=42).fit_resample(X_train, y_train)

In [14]:
estimators = [('bst', XGBClassifier(max_iter=10000, multi='softmax')), ('SGDC',SGDClassifier(max_iter=10000, random_state=42)), ('knn', KNeighborsClassifier(n_neighbors=10)), ('ada', AdaBoostClassifier(n_estimators=500,random_state=0))]

In [15]:
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X_resampled, y_resampled)

StackingClassifier(estimators=[('bst',
                                XGBClassifier(max_iter=10000, multi='softmax')),
                               ('SGDC',
                                SGDClassifier(max_iter=10000, random_state=42)),
                               ('knn', KNeighborsClassifier(n_neighbors=10)),
                               ('ada',
                                AdaBoostClassifier(n_estimators=500,
                                                   random_state=0))],
                   final_estimator=LogisticRegression())

In [16]:
predicted = clf.predict(X_test)
print(metrics.classification_report(predicted, y_test))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         3
           4       0.60      1.00      0.75         3
           5       1.00      0.75      0.86         8

    accuracy                           0.92        24
   macro avg       0.92      0.95      0.92        24
weighted avg       0.95      0.92      0.92        24

